# Using the Rainbow algorithm to learn connect four

The previous notebooks all used Deep Q-Networks (DQN) to learn to play connect four and no real desired behaviour was obtained.
Since rainbow by [Hessel et al](https://doi.org/10.48550/arXiv.1710.02298) is known to outperform DQN in both sample efficiency and overall performance for Atari games, we explore its performance on the connect four game as well. 

<hr><hr>

## Table of Contents

- Contact information
- Checking requirements
  - Correct Anaconda environment
  - Correct module access
  - Correct CUDA access
- Training two Rainbow agents on connect four Gym
  - Building the environment
  - Implementing the Rainbow policy
  - Building agents
  - Function for letting agents learn
  - Function for watching learned agent
  - Doing the experiment
- Discussion

<hr><hr>

## Contact information

| Name             | Student ID | VUB mail                                                  | Personal mail                                               |
| ---------------- | ---------- | --------------------------------------------------------- | ----------------------------------------------------------- |
| Lennert Bontinck | 0568702    | [lennert.bontinck@vub.be](mailto:lennert.bontinck@vub.be) | [info@lennertbontinck.com](mailto:info@lennertbontinck.com) |



<hr><hr>

## Checking requirements

### Correct Anaconda environment

The `rl-project` anaconda environment should be active to ensure proper support. Installation instructions are available on [the GitHub repository of the RL course project and homeworks](https://github.com/pikawika/vub-rl).

In [1]:
####################################################
# CHECKING FOR RIGHT ANACONDA ENVIRONMENT
####################################################

import os
from platform import python_version

print(f"Active environment: {os.environ['CONDA_DEFAULT_ENV']}")
print(f"Correct environment: {os.environ['CONDA_DEFAULT_ENV'] == 'rl-project'}")
print(f"\nPython version: {python_version()}")
print(f"Correct Python version: {python_version() == '3.8.10'}")

Active environment: rl-project
Correct environment: True

Python version: 3.8.10
Correct Python version: True


<hr>

### Correct module access

The following code block will load in all required modules and show if the versions match those that are recommended.

In [4]:
####################################################
# LOADING MODULES
####################################################

# Allow reloading of libraries
import importlib

# Plotting
import matplotlib; print(f"Matplotlib version (3.5.1 recommended): {matplotlib.__version__}")
import matplotlib.pyplot as plt

# Argparser
import argparse

# More data types
import typing
import numpy as np

# Pygame
import pygame; print(f"Pygame version (2.1.2 recommended): {pygame.__version__}")

# Gym environment
import gym; print(f"Gym version (0.21.0 recommended): {gym.__version__}")

# Tianshou for RL algorithms
import tianshou as ts; print(f"Tianshou version (0.4.8 recommended): {ts.__version__}")

# Torch is a popular DL framework
import torch; print(f"Torch version (1.12.0 recommended): {torch.__version__}")

# PPrint is a pretty print for variables
from pprint import pprint

# Our custom connect four gym environment
import sys
sys.path.append('../')
import gym_connect4_pygame.envs.ConnectFourPygameEnvV2 as cfgym
importlib.invalidate_caches()
importlib.reload(cfgym)

# Time for allowing "freezes" in execution
import time;

# Allow for copying objects in a non reference manner
import copy

# Used for updating notebook display
from IPython.display import clear_output

Matplotlib version (3.5.1 recommended): 3.5.1
Pygame version (2.1.2 recommended): 2.1.2
Gym version (0.21.0 recommended): 0.21.0
Tianshou version (0.4.8 recommended): 0.4.8
Torch version (1.12.0 recommended): 1.12.0.dev20220520+cu116


<hr>

### Correct CUDA access

The installation instructions specify how to install PyTorch with CUDA 11.6.
The following code block tests if this was done successfully.

In [5]:
####################################################
# CUDA VALIDATION
####################################################

# Check cuda available
print(f"CUDA is available: {torch.cuda.is_available()}")

# Show cuda devices
print(f"\nAmount of connected devices supporting CUDA: {torch.cuda.device_count()}")

# Show current cuda device
print(f"\nCurrent CUDA device: {torch.cuda.current_device()}")

# Show cuda device name
print(f"Cuda device 0 name: {torch.cuda.get_device_name(0)}")

CUDA is available: True

Amount of connected devices supporting CUDA: 1

Current CUDA device: 0
Cuda device 0 name: NVIDIA GeForce GTX 970


<hr><hr>

## Training two Rainbow agents on connect four Gym

Our connect four gym setup requires two agents, one for each player.
To reduce complexity, agents will always play as the same player, e.g. always as player 1.
It is important to note that connect four is a *solved game*.
According to [The Washington Post](https://www.washingtonpost.com/news/wonk/wp/2015/05/08/how-to-win-any-popular-game-according-to-data-scientists/):

> Connect Four is what mathematicians call a "solved game," meaning you can play it perfectly every time, no matter what your opponent does. You will need to get the first move, but as long as you do so, you can always win within 41 moves.

<hr>

### Building the environment

This code is taken from previous notebooks.
We don't allow invalid moves to make the problem easier for now.

In [6]:
####################################################
# CONNECT FOUR V2 ENVIRONMENT
####################################################

def get_env():
    """
    Returns the connect four gym environment V2 altered for Tianshou and Petting Zoo compatibility.
    Already wrapped with a ts.env.PettingZooEnv wrapper.
    """
    return ts.env.PettingZooEnv(cfgym.env(reward_move= 0, # Set to 1 for reward to make moves (incentivise longer games)
                                          reward_blocking= 1, # Set to 1 for reward to make blocking moves (incentivise defensive games)
                                          reward_invalid= -3,
                                          reward_draw= 15,
                                          reward_win= 25,
                                          reward_loss= -25,
                                          allow_invalid_move= False))
    
    
# Test the environment
env = get_env()
print(f"Observation space: {env.observation_space}")
print(f"\nAction space: {env.action_space}")

# Reset the environment to start from a clean state, returns the initial observation
observation = env.reset()

print("\n Initial player id:")
print(observation["agent_id"])

print("\n Initial observation:")
print(observation["obs"])

print("\n Initial mask:")
print(observation["mask"])

# Clean unused variables
del observation
del env

Observation space: Dict(action_mask:Box([0 0 0 0 0 0 0], [1 1 1 1 1 1 1], (7,), int8), observation:Box([[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]], [[2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2]], (6, 7), int8))

Action space: Discrete(7)

 Initial player id:
player_1

 Initial observation:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]

 Initial mask:
[True, True, True, True, True, True, True]


In [7]:
####################################################
# BLOCKING MOVE CHECK
####################################################

# Check if a reward is received for playing a blocking move

env = get_env()
env.reset()
env.step(action= 0)
print(env.rewards)
env.step(action= 1)
print(env.rewards)
env.step(action= 0)
print(env.rewards)
env.step(action= 1)
print(env.rewards)
env.step(action= 0)
print(env.rewards)
env.step(action= 1)
print(env.rewards)
env.step(action= 1)
print(f"Blocking move made by player 1: {env.rewards}")

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
Blocking move made by player 1: [1, 0]


<hr>

### Implementing the Rainbow policy

The Rainbow policy for the agent is configured and set up below.
This is based on the Atari rainbow example by the [Tianshou doc](https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py).
Tianshou uses a DQN as base class for Rainbow, so we will do this to but adopt both classes to better represent our previous DQN models.

In [8]:
####################################################
# DQN ARCHITECTURE
####################################################

class CNNForRainbow(torch.nn.Module):
    """
    Custom CNN to be used as baseclass for the Rainbow algorithm.
    Extracts "feautures" for the Rainbow algorithm by doing a 4x4 cnn kernel pass and providing 16 filters.
    """
    def __init__(self,
                 state_shape: typing.Sequence[int],
                 device: typing.Union[str, int, torch.device] = 'cuda' if torch.cuda.is_available() else 'cpu'):
        
        # Torch init
        super().__init__()
        
        # Store device to be used
        self.device = device
        
        # The input layer is singular -> we have 1 board vector
        input_channels_cnn = 1
        
        # We output 16 filters per kernel 
        output_channels_cnn = 16
        
        # We store the output dimension of the CNN "feature" layer
        self.output_dim = (state_shape[0] - 3) * (state_shape[1] - 3) * output_channels_cnn
        
        self.net = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels= input_channels_cnn, out_channels= output_channels_cnn, kernel_size= 4, stride= 1), torch.nn.ReLU(inplace=True),
            torch.nn.Flatten(),
        )

    def forward(self,
                obs: typing.Union[np.ndarray, torch.Tensor],
                state: typing.Optional[typing.Any] = None,
                info: typing.Dict[str, typing.Any] = {}):
        # Make a torch instance (from regular vector of board)
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float, device=self.device)
            
        # Tianshou bugs the batch output, reshape to work properly with our torch version
        if (len(np.shape(obs)) != 4):
            obs = obs[:, None, :, :]
        
        # Return what is needed (network output & state)
        return self.net(obs), state


In [9]:
####################################################
# RAINBOW ARCHITECTURE
####################################################

class Rainbow(CNNForRainbow):
    """
    Implementation of the Rainbow algorithm making using of the CNNForRainbow baseclass.
    Default parameters adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
    """

    def __init__(self,
                 state_shape: typing.Sequence[int],
                 action_shape: typing.Sequence[int],
                 device: typing.Union[str, int, torch.device] = 'cuda' if torch.cuda.is_available() else 'cpu',
                 num_atoms: int = 51,
                 is_noisy: bool = True,
                 noisy_std: float = 0.1,
                 is_dueling: bool = True):
        
        # Init CNN feature extraction parent class
        super().__init__(state_shape= state_shape, device= device)
        
        # the amount of actions we have is just the action shape
        self.action_num = np.prod(action_shape)
        
        # Store class specific info
        self.num_atoms = num_atoms
        self._is_dueling = is_dueling

        # Our linear layer depends on wether or not we want to use a noisy environment
        # Noisy implementation based on https://arxiv.org/abs/1706.10295
        def linear(x, y):
            if is_noisy:
                return ts.utils.net.discrete.NoisyLinear(x, y, noisy_std)
            else:
                return torch.nn.Linear(x, y)
            
        # Specify Q and V based on wether or not agent is dueling
        # Setting agent on dueling mode should help generalisation according to rainbow paper
        # NOTE: this uses the output dim from the feature extraction CNN
        self.Q = torch.nn.Sequential(
            linear(self.output_dim, 512), torch.nn.ReLU(inplace=True),
            linear(512, self.action_num * self.num_atoms))
        
        if self._is_dueling:
            self.V = torch.nn.Sequential(
                linear(self.output_dim, 512), torch.nn.ReLU(inplace=True),
                linear(512, self.num_atoms))
            
        # New output dim for this rainbow network
        self.output_dim = self.action_num * self.num_atoms
        

    def forward(self,
                obs: typing.Union[np.ndarray, torch.Tensor],
                state: typing.Optional[typing.Any] = None,
                info: typing.Dict[str, typing.Any] = {}):
        
        # Use our parent CNN based network to get "features"
        obs, state = super().forward(obs)
        
        # Get our Rainbow specific values
        q = self.Q(obs)
        q = q.view(-1, self.action_num, self.num_atoms)
        
        if self._is_dueling:
            v = self.V(obs)
            v = v.view(-1, 1, self.num_atoms)
            logits = q - q.mean(dim=1, keepdim=True) + v
        else:
            logits = q
        
        # We need to go from our logits to an accepted dimension of probability outputs
        probs = logits.softmax(dim=2)
        
        return probs, state

In [10]:
####################################################
# RAINBOW POLICY
####################################################

def rainbow_policy(state_shape: tuple,
                   action_shape: tuple,
                   optim: typing.Optional[torch.optim.Optimizer] = None,
                   learning_rate: float =  0.0000625,
                   gamma: float = 0.9,
                   n_step: int = 3, # Number of steps to look ahead
                   num_atoms: int = 51,
                   is_noisy: bool = True,
                   noisy_std: float = 0.1,
                   is_dueling: bool = True,
                   target_update_freq: int = 500):
    """
    Implementation of the Rainbow policy.
    Default parameters adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
    """
    
    # Use cuda device if possible
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Rainbow network to be used by policy
    net = Rainbow(state_shape= state_shape,
                  action_shape= action_shape,
                  device= device,
                  num_atoms= num_atoms,
                  is_noisy= is_noisy,
                  noisy_std= noisy_std,
                  is_dueling= is_dueling).to(device)
    
    # Default optimizer is an adam optimizer with the argparser learning rate
    if optim is None:
        optim = torch.optim.Adam(net.parameters(), lr= learning_rate)
        
    # Our agents Rainbow policy
    return ts.policy.RainbowPolicy(model= net,
                                   optim= optim,
                                   discount_factor= gamma,
                                   num_atoms= num_atoms,
                                   estimation_step= n_step,
                                   target_update_freq= target_update_freq).to(device)
    
    

<hr>

### Building agents

Identical to the previous notebook.

In [11]:
####################################################
# AGENT CREATION
####################################################

def get_agent_manager(agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
                      agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
                      optim: typing.Optional[torch.optim.Optimizer] = None):
    """
    Gets a multi agent policy manager, optimizer and player ids for the connect four V2 gym environment.
    Per default this returns 
        - Multi agent manager for 2 agents using Rainbow
        - Adam optimizer
        - ['player_1', 'player_2'] from the connect four environment
    """
    
    # Get the environment to play in (Connect four gym V2)
    env = get_env()
    
    # Get the observation space from the environment, depending on typo of space (ternary operator)
    observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
    
    # Set the arguments
    state_shape = observation_space.shape or observation_space.n
    action_shape = env.action_space.shape or env.action_space.n
    
    # Configure agent player 1 to be a Rainbow if no policy is passed.
    if agent_player1 is None:
        # Our agent1 uses a Rainbow policy
        agent_player1 = rainbow_policy(state_shape= state_shape,
                                       action_shape= action_shape,
                                       optim= optim)
    
    # Configure agent player 2 to be a Rainbow if no policy is passed.
    if agent_player2 is None:
        # Our agent1 uses a Rainbow policy
        agent_player2 = rainbow_policy(state_shape= state_shape,
                                       action_shape= action_shape,
                                       optim= optim)

    # Default order of the agents
    agents = [agent_player1, agent_player2]
        
    # Create the multi agent policy
    policy = ts.policy.MultiAgentPolicyManager(agents, env)
    
    # Return our policy, optimizer and the available agents in the environment
    # Per default: 
    #   - Multi agent manager for 2 agents using Rainbow
    #   - Adam optimizer
    #   - ['player_1', 'player_2'] from the connect four environment
    
    return policy, optim, env.agents

<hr>

### Function for letting agents learn

Identical to the previous notebook.

In [12]:
####################################################
# AGENT TRAINING
####################################################

def train_agent(filename: str = "rainbow_vs_rainbow",
                agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
                agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
                optim: typing.Optional[torch.optim.Optimizer] = None,
                training_env_num: int = 10,
                testing_env_num: int = 10,
                buffer_size: int = 100000,
                batch_size: int = 32, 
                epochs: int = 50,
                step_per_epoch: int = 100000,
                step_per_collect: int = 10, # Should be multiple of the test/training envs
                update_per_step: float = 0.1,
                testing_eps: float = 0.005,
                training_eps_init: float = 1,
                training_eps_final: float = 0.05):
    """
    Trains two agents in the connect four V2 environment and saves their best model and logs.
    Returns:
        - result from offpolicy_trainer
        - final version of agent 1
        - final version of agent 2
    Defaults adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
    """

    # ======== notebook specific =========
    notebook_version = '9' # Used for foldering logs and models

    # ======== environment setup =========
    train_envs = ts.env.DummyVectorEnv([get_env for _ in range(training_env_num)])
    test_envs = ts.env.DummyVectorEnv([get_env for _ in range(testing_env_num)])
    
    # set the seed for reproducibility
    np.random.seed(1998)
    torch.manual_seed(1998)
    train_envs.seed(1998)
    test_envs.seed(1998)

    # ======== agent setup =========
    # Gets our agents from the previously made function
    # Per default: 
    #   - Multi agent manager for 2 agents using Rainbow
    #   - Adam optimizer
    #   - ['player_1', 'player_2'] from the connect four environment
    policy, optim, agents = get_agent_manager(agent_player1=agent_player1,
                                              agent_player2=agent_player2,
                                              optim=optim)

    # ======== collector setup =========
    # Make a collector for the training environments
    buffer= ts.data.VectorReplayBuffer(total_size= buffer_size,
                                       buffer_num=len(train_envs))
    
    train_collector = ts.data.Collector(policy= policy,
                                        env= train_envs,
                                        buffer= buffer,
                                        exploration_noise= True)
    
    # Make a collector for the testing environments
    test_collector = ts.data.Collector(policy= policy,
                                       env= test_envs,
                                       exploration_noise= True)
    
    # ======== ensure folders exist =========
    if not os.path.exists(os.path.join('./logs', 'paper_notebooks', notebook_version, filename)):
        os.makedirs(os.path.join('./logs', 'paper_notebooks', notebook_version, filename))
    if not os.path.exists(os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename)):
        os.makedirs(os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename))

    # ======== tensorboard logging setup =========
    # Allows to save the training progress to tensorboard compatable logs
    log_path = os.path.join('./logs', 'paper_notebooks', notebook_version, filename)
    writer = torch.utils.tensorboard.SummaryWriter(log_path)
    logger = ts.utils.TensorboardLogger(writer)

    # ======== callback functions used during training =========
    # We want to save our best policy
    def save_best_fn(policy):
        """
        Callback to save the best model
        """
        # Save best agent 1
        model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'best_policy_agent1.pth')
        torch.save(policy.policies[agents[0]].state_dict(), model_save_path)
        
        # Save best agent 2
        model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'best_policy_agent2.pth')
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)
        
        # Save agent2

    def stop_fn(mean_rewards):
        """
        Callback to stop training when we've reached the win rate
        """
        return False # Not implemented

    def train_fn(epoch, env_step):
        """
        Callback before training, sets the training epsilon in a decaying manner.
        Adopted from: https://github.com/thu-ml/tianshou/blob/master/examples/atari/atari_rainbow.py
        """        
        # Nature DQN setting to have a "linear decaying epsilon" for the first one million iterations
        if env_step <= 1000000:
            training_eps = training_eps_init - env_step / 1000000 * (training_eps_init - training_eps_final)
        else:
            training_eps = training_eps_final
            
        # Set epsilon
        policy.policies[agents[0]].set_eps(training_eps)
        policy.policies[agents[1]].set_eps(training_eps)

    def test_fn(epoch, env_step):
        """
        Callback beore testing, sets the testing epsilon.
        """        
        # Before testing we want to configure the epsilon for the agents
        # In general more greedy than the train case but not
        #   to avoid getting stuck on invalid moves
        policy.policies[agents[0]].set_eps(testing_eps)
        policy.policies[agents[1]].set_eps(testing_eps)

    def reward_metric(rews):
        """
        Callback for reward collection.
        Currently the reward is the sum of both agents.
        """        
        # We are interested in having a high total total reward,
        #   as this would mean equally good agents.
        return rews[:, 0] + rews[:, 1]

    # ======== Training =========
    # off policy training
    result = ts.trainer.offpolicy_trainer(policy= policy,
                                          train_collector= train_collector,
                                          test_collector= test_collector,
                                          max_epoch= epochs,
                                          step_per_epoch= step_per_epoch,
                                          step_per_collect= step_per_collect,
                                          episode_per_test= testing_env_num,
                                          batch_size= batch_size,
                                          train_fn= train_fn,
                                          test_fn= test_fn,
                                          stop_fn= stop_fn,
                                          save_best_fn= save_best_fn,
                                          update_per_step= update_per_step,
                                          logger= logger,
                                          test_in_train= False,
                                          reward_metric= reward_metric)
    
    # Save final agent 1
    model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'final_policy_agent1.pth')
    torch.save(policy.policies[agents[0]].state_dict(), model_save_path)

    # Save final agent 2
    model_save_path = os.path.join('./saved_variables', 'paper_notebooks', notebook_version, filename, 'final_policy_agent2.pth')
    torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

    return result, policy.policies[agents[0]], policy.policies[agents[1]]


<hr>

### Function for watching learned agent

Identical to the previous notebook.

In [13]:
####################################################
# WATCHING THE LEARNED POLICY IN ACTION
####################################################

def watch(numer_of_games: int = 3,
          agent_player1: typing.Optional[ts.policy.BasePolicy] = None,
          agent_player2: typing.Optional[ts.policy.BasePolicy] = None,
          test_epsilon: float = 0.005, # For the watching we act completely greedy but low random for not getting stuck on invalid move
          render_speed: float = 0.15, # Amount of seconds to update frame/ do a step
          ) -> None:
    
    # Get the connect four V2 environment (must be a list)
    env= ts.env.DummyVectorEnv([get_env])
    
    # Get the agents from the trained agents
    policy, optim, agents = get_agent_manager(agent_player1= agent_player1,
                                              agent_player2= agent_player2)
    
    # Evaluate the policy
    policy.eval()
    
    # Set the testing policy epsilon for our agents
    policy.policies[agents[0]].set_eps(test_epsilon)
    policy.policies[agents[1]].set_eps(test_epsilon)
    
    # Collect the test data
    collector = ts.data.Collector(policy= policy,
                                  env= env,
                                  exploration_noise= True)
    
    # Render games in human mode to see how it plays
    result = collector.collect(n_episode= numer_of_games, render= render_speed)
    
    # Close the environment aftering collecting the results
    # This closes the pygame window after completion
    env.close()
    
    # Get the rewards and length from the test trials
    rewards, length = result["rews"], result["lens"]
    
    # Print the final reward for the first agent
    print(f"Average steps of game:  {length.mean()}")
    print(f"Final mean reward agent 1: {rewards[:, 0].mean()}, std: {rewards[:, 0].std()}")
    print(f"Final mean reward agent 2: {rewards[:, 1].mean()}, std: {rewards[:, 1].std()}")

<hr>

### Doing the experiment

We now do the experiment with using our previously created functions.
We update some parameter settings to find if we can improve our DQN agents.

In [ ]:
####################################################
# EXPERIMENT: TRAINING AGENTS
####################################################

# Get the environment settings
env = get_env()
observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
state_shape = observation_space.shape or observation_space.n
action_shape = env.action_space.shape or env.action_space.n

# Configure the agents
agent1 = rainbow_policy(state_shape= state_shape,
                        action_shape= action_shape,
                        learning_rate=  0.0001, # Increased from 0000625
                        gamma= 0.8, # Decreased from 0.9
                        n_step= 3, # Number of steps to look ahead
                        num_atoms= 51,
                        is_noisy= True,
                        noisy_std= 0.1,
                        is_dueling= True)


agent2 = rainbow_policy(state_shape= state_shape,
                        action_shape= action_shape,
                        learning_rate=  0.0001, # Increased from 0000625
                        gamma= 0.8, # Decreased from 0.9
                        n_step= 3, # Number of steps to look ahead
                        num_atoms= 51,
                        is_noisy= True,
                        noisy_std= 0.1,
                        is_dueling= True)

# Train the agent
off_policy_traininer_results, final_agent_player1, final_agent_player2 = train_agent(filename= "rainbow_vs_rainbow_blocking_reward_fewer_iterations",
                                                                                     epochs= 500, # Default 50
                                                                                     agent_player1= agent1,
                                                                                     agent_player2= agent2,
                                                                                     training_env_num= 10,
                                                                                     testing_env_num= 10,
                                                                                     buffer_size= 10000, # Default 100000
                                                                                     batch_size= 64, # Default 32
                                                                                     step_per_epoch= 10000, # Default 100000
                                                                                     step_per_collect= 10,
                                                                                     update_per_step= 0.1,
                                                                                     testing_eps= 0.005,
                                                                                     training_eps_init= 1,
                                                                                     training_eps_final= 0.05)

Epoch #1: 100001it [07:16, 229.01it/s, env_step=100000, len=17, n/ep=0, n/st=10, player_1/loss=2.774, player_2/loss=2.918, rew=0.00]


Epoch #1: test_reward: 0.400000 ± 1.200000, best_reward: 0.500000 ± 1.500000 in #0


Epoch #2: 100001it [07:37, 218.54it/s, env_step=200000, len=32, n/ep=0, n/st=10, player_1/loss=2.794, player_2/loss=2.900, rew=35.00]


Epoch #2: test_reward: 41.200000 ± 5.400000, best_reward: 41.200000 ± 5.400000 in #2


Epoch #3: 100001it [05:24, 307.80it/s, env_step=300000, len=9, n/ep=0, n/st=10, player_1/loss=2.770, player_2/loss=2.903, rew=0.00]


...

Epoch #41: 100001it [05:36, 296.95it/s, env_step=4100000, len=16, n/ep=1, n/st=10, player_1/loss=0.821, player_2/loss=0.980, rew=14.00]


Epoch #41: test_reward: 185.600000 ± 61.727142, best_reward: 233.000000 ± 0.000000 in #25


Epoch #42: 100001it [05:33, 300.26it/s, env_step=4200000, len=13, n/ep=0, n/st=10, player_1/loss=0.756, player_2/loss=0.883, rew=14.50]


Epoch #42: test_reward: 73.200000 ± 66.010302, best_reward: 233.000000 ± 0.000000 in #25


Epoch #43:  79%|7| 79140/100000 [04:26<01:11, 292.37it/s, env_step=4279140, len=29, n/ep=0, n/st=10, player_1/loss=0

In [ ]:
####################################################
# EXPERIMENT: VIEWING THE BEST LEARNED POLICY
####################################################

# Get the environment settings
env = get_env()
observation_space = env.observation_space['observation'] if isinstance(env.observation_space, gym.spaces.Dict) else env.observation_space
state_shape = observation_space.shape or observation_space.n
action_shape = env.action_space.shape or env.action_space.n

# Configure the best agent
best_agent1 = rainbow_policy(state_shape= state_shape,
                             action_shape= action_shape)
best_agent1.load_state_dict(torch.load("./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward/best_policy_agent1.pth"))
best_agent1.set_eps(0)


best_agent2 = rainbow_policy(state_shape= state_shape,
                             action_shape= action_shape)
best_agent2.load_state_dict(torch.load("./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward/best_policy_agent2.pth"))
best_agent2.set_eps(0)

# Watch the best agent at work
watch(numer_of_games= 3,
      render_speed= 0.3,
      agent_player1= best_agent1,
      agent_player2= best_agent2)



In [ ]:
####################################################
# EXPERIMENT: VIEWING THE LAST LEARNED POLICY
####################################################

# Configure the final agent
final_agent_player1 = rainbow_policy(state_shape= state_shape,
                                     action_shape= action_shape)
final_agent_player1.load_state_dict(torch.load("./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward/final_policy_agent1.pth"))
best_agent1.set_eps(0)

final_agent_player2 = rainbow_policy(state_shape= state_shape,
                                     action_shape= action_shape)
final_agent_player2.load_state_dict(torch.load("./saved_variables/paper_notebooks/9/rainbow_vs_rainbow_blocking_reward/final_policy_agent2.pth"))
best_agent2.set_eps(0)

# Watch the best agent at work
watch(numer_of_games= 3,
      render_speed= 0.3,
      agent_player1= final_agent_player1,
      agent_player2= final_agent_player2)



<hr><hr>

## Discussion

When a reward for making moves is given, the agents learn to play together rather then against each other. Just as was the case for DQN. This can be seen by looking at the gameplay of the best reward agents where a tie is reached.
> Best reward: 594.700000 ± 344.331541 in epoch 64

Afterwards we did an experiment with the "default" parameters, which does ten times as many iterations per epoch. The found results for the model using a reward for each move is a perfect synced play between agents of getting ties. Thus, this would have been identical as just doing more epochs.
> Best reward: 932.000000 ± 0.000000 in epoch 37

Since the agent optimizes to the rewards, it is more logical to give it a reward when blocking a winning move rather then just for any move made.
This will incentive defensive behaviour which is what we want indeed. 
Since testing only happens on epochs and thus saving the model only happens after an epoch, we beleive it might be better to do shorter epochs but more epochs. For the default iteration and epoch count we got:
> Best reward: 233.000000 ± 0.000000 in epoch 25

After this we used rewards for blocking moves but using 10 times as many epochs for 10 times fewer iterations per epoch. The batch size was also increased to 64 and the learning rate was increased to 0,0001. The gamma was decreased to 0.8. All of this was done to combat the harder network to train and to have more checkpoints for testing for the best model. We got:
> Todo

It is noted that comparing results is difficult since the reward strategy is different between the experiments


In [ ]:
####################################################
# CLEAN VARIABLES
####################################################

del action_shape
del agent1
del agent2
del best_agent1
del best_agent2
del env
del final_agent_player1
del final_agent_player2
del observation_space
del off_policy_traininer_results
del state_shape
